## Feature Extraction

#### Importations

In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from skimage.io import imread
from skimage.io import imsave
from skimage.color import rgb2gray
from skimage.transform import resize
from skimage.transform import rotate
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
from time import time 
from scipy.ndimage.measurements import center_of_mass
from scipy.stats import gaussian_kde
from scipy import ndimage
from skimage.morphology import binary_opening
import cv2
from skimage.color import rgb2hsv

#### Réduction de taille
Pour diminuer le temps de calcul des algorithmes, on réduit la taille des images.

In [10]:
### On change la taille des Images####
import cv2
t = time()
feat_df = pd.read_csv('features.csv', sep=';')

# On parcourt toutes les images du dossier
for x in feat_df['ImageId'].values:
    filename = 'im/{}.jpg'.format(x)
    filename_S = 'im/{}_Segmentation.jpg'.format(x)
    # Chargement des images
    image = imread(filename)
    image_S = imread(filename_S, as_grey = True)
    (h,w,d) = image.shape
    # On veut que l'image garde la même forme, on fixe donc la largeur ou la longueur, selon les cas et on modifie 
    # proportionelement l'autre dimension
    if w>d:
        new_h, new_w = 300,int(w/h*300)
    else :
        new_w, new_h = 300,int(h/w*300)
    # on applique le changement de taille et on l'enregistre
    image_downsampled = resize(image,(new_h, new_w), mode='reflect')
    image_Segmentation_downsampled = resize(image_S,(new_h, new_w), mode='reflect')
    imsave('im_resized/{}_Segmentation_resized.jpg'.format(x), image_Segmentation_downsampled)
    imsave('im_resized/{}_resized.jpg'.format(x), image_downsampled)
    
print('Temps de calcul : ', time()-t)

/Users/matthis/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/Users/matthis/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: im_resized/IM_000720_Segmentation_resized.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)


Temps de calcul :  311.83210706710815


La première erreur soulevée est due à la commande "int(h/w*300)" mais cela est nécéssaire car les coordonées de l'image ne peuvent pas être des réels. La seconde erreur vient du fait que la segmentation de l'image 720 est entièrement blanche, c'est à dire sans aucun contraste

#### Asymétrie des formes

Correspond au Paragraphe 2.3.1 de l'article https://www.sciencedirect.com/science/article/pii/S0933365713001589

In [17]:
#on calcule le temps car l'exécution est très longue
t = time()
# Toutes les 50 itérations de m, on va afficher l'avancée de l'algorithme
m=0
for x in feat_df['ImageId'].values:
    m+=1
    filename = 'im_resized/{}_resized.jpg'.format(x)
    filename_S = 'im_resized/{}_Segmentation_resized.jpg'.format(x)
    image = imread(filename)
    image_S = imread(filename_S)
    # Je seuille l'image car lors de l'enregistrement de la segmentation dans un fichier JPEG, puis la relecture,
    # il y a des valeurs légèrement supérieur à 0. Or, on veut une image binaire. Cela n'est pas du à l'interpolation
    # lors du sous-echantillonage.
    ret, image_S = cv2.threshold(image_S, 177, 255, cv2.THRESH_BINARY)
    image_S = image_S.astype(bool)
        
    # On convertit l'image en niveau de gris
    gray_image = rgb2gray(image)*255
    gray_image = np.array(gray_image, np.int32)
    n,p = gray_image.shape
    
    # On calcule le centre de masse du masque de l'image.
    i_center, j_center = center_of_mass(image_S)
    i_center = int(i_center)
    j_center = int(j_center)
    #toutes les valeurs hors de la lésion doivent être nulles
    masked_im = np.array(gray_image*image_S, np.int32)
    # On cherche les valeurs minimales de DS1 et DS2 pour tous les axes orthogonaux 
    # tournés par pas de 10 degrés    
    #on choisit des valeurs arbitrairement très grandes pour être sûr qu'un angle ait des valeur plus petites pour DS1 et DS2
    averageDS = 100
        
    #on recentre et agrandie l'image pour que la lésion soit toujours dans l'image lors des rotations
    padX = [p - j_center, j_center]
    padY = [n - i_center, i_center]
    imgP = np.pad(masked_im, [padY, padX], 'constant')
        
    for angle in [10*k for k in range(18)]:        
        #rotated_image = rotate(masked_im, angle, center=(j_center, i_center))
        rotated_image = ndimage.rotate(imgP, angle, output=np.int32 ,reshape=False)
        #on ne veut pas que les valeur de l'image dépassent 0 ou 255
        rotated_image = np.clip(rotated_image, 0, 255)
            
        nbis, pbis = rotated_image.shape
            
        # on calcule les différences des parties de la lésion 
        DS1 = np.sum(np.abs(rotated_image[:,:pbis//2]-np.fliplr(rotated_image[:,pbis//2:pbis])))
        DS2 = np.sum(np.abs(rotated_image[:nbis//2,:]-np.flipud(rotated_image[nbis//2:nbis, :])))
        
        #On normalise les valeurs selon la taille du masque
        DS1 /= np.sum(np.sum(image_S*255))
        DS2 /= np.sum(np.sum(image_S*255))
        #on choisit la combinaison qui minimise la moyenne des deux
        if((DS1+DS2)/2 < averageDS):
            minDS1 = DS1
            minDS2 = DS2
            averageDS = (DS1+DS2)/2
        
    if(m%50 ==0):
        print("Image: ", m, ", Temps : ", time()-t)
    feat_df.loc[feat_df['ImageId'] == x, 'f1'] = minDS1
    feat_df.loc[feat_df['ImageId'] == x, 'f2'] = minDS2
print(time()-t)
feat_df.to_csv('features.csv', index=None, sep=';', mode='w')

Image:  50 , Temps :  98.58412289619446
Image:  100 , Temps :  199.68367075920105
Image:  150 , Temps :  296.52192187309265
Image:  200 , Temps :  391.15829586982727
Image:  250 , Temps :  483.26485300064087
Image:  300 , Temps :  581.7804698944092
Image:  350 , Temps :  671.7664840221405
Image:  400 , Temps :  764.7944917678833
Image:  450 , Temps :  858.178827047348
Image:  500 , Temps :  952.8784098625183
Image:  550 , Temps :  1046.7902228832245
Image:  600 , Temps :  1140.2107746601105
Image:  650 , Temps :  1228.828293800354
Image:  700 , Temps :  1319.4477019309998
Image:  750 , Temps :  1416.4046227931976
Image:  800 , Temps :  1514.776107788086
Image:  850 , Temps :  1609.883752822876
Image:  900 , Temps :  1701.6312539577484
1701.642431974411


#### Asymétrie des couleurs

Correspond au Paragraphe 2.3.2 de l'article https://www.sciencedirect.com/science/article/pii/S0933365713001589

In [22]:
t = time()
m=0
for x in feat_df['ImageId'].values:
    m+=1
    filename = 'im_resized/{}_resized.jpg'.format(x)
    filename_S = 'im_resized/{}_Segmentation_resized.jpg'.format(x)
    image = imread(filename)
    image_S = imread(filename_S)
    ret, image_S = cv2.threshold(image_S, 177, 255, cv2.THRESH_BINARY)
    image_S = image_S.astype(bool)
    
    #On convertit l'image en niveau de gris
    gray_image = np.array(rgb2gray(image)*255, dtype = np.int32)
    n,p = gray_image.shape
    
    #On calcule le centre de masse du masque de l'image.
    i_center, j_center = center_of_mass(image_S)
    i_center, j_center = int(i_center), int(j_center)
    averageDC = 100
    
    #on applique le masque de segmentation à l'image
    masked_im = gray_image * image_S
    
    #on recentre et agrandie l'image pour que la lésion soit toujours dans l'image lors des rotations
    padX = [p - j_center, j_center]
    padY = [n - i_center, i_center]
    imgP = np.pad(masked_im, [padY, padX], 'constant')
    
    #on tourne de 0 à 170 degrés
    for angle in [10*k for k in range(18)]:
        # rotation de l'image
        rotated_image = ndimage.rotate(imgP, angle, reshape=False)
        
        nbis, pbis = rotated_image.shape
        
        # on séléctionnes les quatres quarts de l'image
        values_above_j = rotated_image[:, :pbis//2]
        values_under_j = rotated_image[:, pbis//2:pbis]
        
        values_above_i = rotated_image[:nbis//2, :]
        values_under_i = rotated_image[nbis//2:nbis, :]
        
        
        #On enlève les valeurs nulles càd celles qui sont hors du masque et on met tout dans une seule liste
        values_above_j = values_above_j[values_above_j > 0]
        values_under_j = values_under_j[values_under_j > 0]
        values_above_i = values_above_i[values_above_i > 0]
        values_under_i = values_under_i[values_under_i > 0]
        
        # On calcule la distribution des ensembles
        distribution_a_j = gaussian_kde(values_above_j)
        distribution_u_j = gaussian_kde(values_under_j)
        
        distribution_a_i = gaussian_kde(values_above_i)
        distribution_u_i = gaussian_kde(values_under_i)
        
        
        A = np.linspace(0,255, 256)
        # On calcule pour chaque valeur possible de pixel la différence de la distribution des différentes parties
        DC1 = np.sum(np.abs(distribution_a_j(A) - distribution_u_j(A)))
        DC2 = np.sum(np.abs(distribution_a_i(A) - distribution_u_i(A)))
        
        #On normalise les valeurs selon la taille du masque
        DC1 /= np.sum(np.sum(image_S))
        DC2 /= np.sum(np.sum(image_S))
        
        #on choisit la combinaison qui minimise la moyenne des deux
        if((DC1+DC2)/2 < averageDC):
            minDC1 = DC1
            minDC2 = DC2
            averageDC = (DC1+DC2)/2
            
    if(m%50 ==0):
        print("Image: ", m, ", Temps : ", time()-t)
    feat_df.loc[feat_df['ImageId'] == x, 'f3'] = minDC1
    feat_df.loc[feat_df['ImageId'] == x, 'f4'] = minDC2

feat_df.to_csv('features.csv', index=None, sep=';', mode='w')    
print(time()-t)

Image:  50 , Temps :  392.6701691150665
Image:  100 , Temps :  848.6166729927063
Image:  150 , Temps :  1283.4140889644623
Image:  200 , Temps :  1729.4835178852081
Image:  250 , Temps :  2115.733841896057
Image:  300 , Temps :  2550.153974056244
Image:  350 , Temps :  3000.3810329437256
Image:  400 , Temps :  3446.1576159000397
Image:  450 , Temps :  3919.5309607982635
Image:  500 , Temps :  4379.600751876831
Image:  550 , Temps :  4822.993721008301
Image:  600 , Temps :  5213.594225883484
Image:  650 , Temps :  5690.094100952148
Image:  700 , Temps :  6073.771212100983
Image:  750 , Temps :  6488.5040719509125
Image:  800 , Temps :  6928.637995958328
Image:  850 , Temps :  7332.421272039413
Image:  900 , Temps :  7766.310403108597
7766.364050865173


#### Asymétrie couleur - forme

Correspond au Paragraphe 2.3.3 de l'article https://www.sciencedirect.com/science/article/pii/S0933365713001589

In [24]:
t = time()
k=0
for x in feat_df['ImageId'].values:
    k+=1
    filename = 'im_resized/{}_resized.jpg'.format(x)
    filename_S = 'im_resized/{}_Segmentation_resized.jpg'.format(x)
    image = imread(filename)
    image_S = imread(filename_S)
    ret, image_S = cv2.threshold(image_S, 178, 255, cv2.THRESH_BINARY)     
    mask = image_S.astype(bool)
    #On convertit l'image en niveau de gris
    gray_image = np.array(rgb2gray(image)*255, np.int32)
    n,p = gray_image.shape

    #On calcule le centre de masse du masque de l'image.
    i_center, j_center = center_of_mass(image_S)
    i_center, j_center = np.int(i_center), np.int(j_center)
    # on enlève toutes les valeurs nulles car ce sont celles en dehors de la lesion
    lesion_values = gray_image*mask
    lesion_values = lesion_values[lesion_values>0]
    
    percentiles = np.percentile(lesion_values, np.linspace(10, 90, 9), interpolation = 'lower')
    #contient les ditances euclidiennes entres les nouveaux centres de masse
    v=[]
    for per in percentiles:

        #centre de masse du masque des valeurs superieur au percentile per
        new_i, new_j = center_of_mass(gray_image*mask > per)
        #on ajoute la ditance eucliedienne
        v.append(np.sqrt((i_center-new_i)**2+(j_center-new_j)**2))

    # on normalise v par le rayon d'un cerlce d'aire égale à celle de la segmentation Image_S
    v = v/(np.sqrt(sum(sum(image_S))/np.pi))

    feat_df.loc[feat_df['ImageId'] == x, 'f5'] = np.mean(v)
    feat_df.loc[feat_df['ImageId'] == x, 'f6'] = np.std(v)

    if(k%50==0):
        print("Image: ", k, ", Temps : ", time()-t)

feat_df.to_csv('features.csv', index=None, sep=';', mode='w')    
print(time()-t)

Image:  50 , Temps :  1.7509431838989258
Image:  100 , Temps :  3.9118881225585938
Image:  150 , Temps :  5.611047983169556
Image:  200 , Temps :  7.484312057495117
Image:  250 , Temps :  9.32894492149353
Image:  300 , Temps :  11.174020290374756
Image:  350 , Temps :  12.848567247390747
Image:  400 , Temps :  14.647001266479492
Image:  450 , Temps :  16.417936325073242
Image:  500 , Temps :  18.09332823753357
Image:  550 , Temps :  19.992983102798462
Image:  600 , Temps :  22.17376208305359
Image:  650 , Temps :  23.83597707748413
Image:  700 , Temps :  25.27400016784668
Image:  750 , Temps :  26.73813509941101
Image:  800 , Temps :  28.269097089767456
Image:  850 , Temps :  29.847776174545288
Image:  900 , Temps :  31.42053198814392
31.463662147521973


#### Geométrie

Correspond au Paragraphe 2.3.8 de l'article https://www.sciencedirect.com/science/article/pii/S0933365713001589

In [25]:
t = time()
k=1
for x in feat_df['ImageId'].values:
    filename = 'im_resized/{}_resized.jpg'.format(x)
    filename_S = 'im_resized/{}_Segmentation_resized.jpg'.format(x)
    image = imread(filename)
    image_S = imread(filename_S)       
    ret, image_S = cv2.threshold(image_S, 178, 255, cv2.THRESH_BINARY)          
    image_S = image_S.astype(bool)
    n,p,d = image.shape
    
    #on met toutes les valeurs hors du masque à 0
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    masked_gray_image = gray_image*image_S
        
    #on choisit les valeurs non nulles car il s'agit de celles qui sont dans le masque (la lésion)
    im_values = masked_gray_image[masked_gray_image>0]
    
    #on calcule les quantiles à 0.25, 0.5 et 0.75
    percentiles = np.percentile(im_values, [25, 50, 75], interpolation='lower')
    
    #liste qui contient le nombre de composantes connexes
    num_conx = []
    for per in percentiles:
        ret, thresh = cv2.threshold(masked_gray_image, per, 255, cv2.THRESH_BINARY)
        #on applique une ouverture au masque des valeurs inférieures au percentile donné
        im_p1 = binary_opening(255 - thresh, [[0,1,0], [1,1,1], [0,1,0]])
        im_p1 = im_p1*image_S
        im_p1 = np.uint8(im_p1)
        #on cherche les composantes connexes
        output = cv2.connectedComponents(im_p1, 8)
        #on ajoute le nombre de composantes connexes
        num_conx.append(output[0]-1)
    
    feat_df.loc[feat_df['ImageId'] == x, 'f15'] = num_conx[0]
    feat_df.loc[feat_df['ImageId'] == x, 'f16'] = num_conx[1]
    feat_df.loc[feat_df['ImageId'] == x, 'f17'] = num_conx[2]
    k+=1
    if(k%50 == 0):
        print("Image: ", k, ", Temps : ", time()-t)
        
feat_df.to_csv('features.csv', index=None, sep=';', mode='w')    
print(time()-t)

Image:  50 , Temps :  1.7220351696014404
Image:  100 , Temps :  3.2678439617156982
Image:  150 , Temps :  5.051492929458618
Image:  200 , Temps :  6.717071056365967
Image:  250 , Temps :  8.248589038848877
Image:  300 , Temps :  10.21944785118103
Image:  350 , Temps :  11.780915975570679
Image:  400 , Temps :  13.415989875793457
Image:  450 , Temps :  15.025693893432617
Image:  500 , Temps :  16.51211905479431
Image:  550 , Temps :  17.977150917053223
Image:  600 , Temps :  19.44136381149292
Image:  650 , Temps :  21.09029793739319
Image:  700 , Temps :  22.54696488380432
Image:  750 , Temps :  24.018200874328613
Image:  800 , Temps :  25.489809036254883
Image:  850 , Temps :  26.96040391921997
Image:  900 , Temps :  28.42581009864807
28.49964213371277


#### Moyenne, variance, 5e percentile et 95e percentile en HSV
D'après :  https://ieeexplore.ieee.org/document/918473/

In [28]:
t = time()
for x in feat_df['ImageId'].values:
    filename = 'im_resized/{}_resized.jpg'.format(x)
    filename_S = 'im_resized/{}_Segmentation_resized.jpg'.format(x)
    image = imread(filename)
    image_S = imread(filename_S)
    ret, image_Segmentation_downsampled = cv2.threshold(image_Segmentation_downsampled, 178, 255, cv2.THRESH_BINARY)          
    mask = image_S.astype(bool)
    n,p,d = image.shape
    
    #on convertit l'image dans le domaine HSV
    image = rgb2hsv(image)
    # on normalise les images
    unique, count = np.unique(image[:,:,0], return_counts= True)
    h_image = image[:,:,0] - unique[np.argmax(count)]
    #on sélectionne les valeurs hors du masque (la peau qui n'est pas la lésion)
    out_skin = image[:,:,2]*(1-mask)
    out_skin = out_skin[out_skin>0]
    v_image = image[:,:,2]-np.mean(out_skin)
    #les valeurs nulles sont celles qui sont hors du masque donc on les enlève
    v_lesion_values = v_image[v_image*mask !=0]
    h_lesion_values = h_image[h_image*mask !=0]
    
    feat_df.loc[feat_df['ImageId'] == x, 'f7'] = np.percentile(v_lesion_values, 5, interpolation = 'lower')
    feat_df.loc[feat_df['ImageId'] == x, 'f8'] = np.percentile(v_lesion_values, 95, interpolation = 'lower')
    feat_df.loc[feat_df['ImageId'] == x, 'f9'] = np.mean(v_lesion_values)
    feat_df.loc[feat_df['ImageId'] == x, 'f10'] = np.var(v_lesion_values)
    
    feat_df.loc[feat_df['ImageId'] == x, 'f11'] = np.percentile(h_lesion_values, 5, interpolation = 'lower')
    feat_df.loc[feat_df['ImageId'] == x, 'f12'] = np.percentile(h_lesion_values, 95, interpolation = 'lower')
    feat_df.loc[feat_df['ImageId'] == x, 'f13'] = np.mean(h_lesion_values)
    feat_df.loc[feat_df['ImageId'] == x, 'f14'] = np.var(h_lesion_values)

feat_df.to_csv('features.csv', index=None, sep=';', mode='w')
print("temps d'execution :", time()-t, "secondes")

temps d'execution : 77.58569693565369 secondes


#### Moyenne, variance, percentiles de RGB

In [30]:
t = time()
for x in feat_df['ImageId'].values:
    filename = 'im_resized/{}_resized.jpg'.format(x)
    filename_S = 'im_resized/{}_Segmentation_resized.jpg'.format(x)
    image = imread(filename)
    image_S = imread(filename_S)
    ret, image_Segmentation_downsampled = cv2.threshold(image_Segmentation_downsampled, 178, 255, cv2.THRESH_BINARY)          
    mask = image_S.astype(bool)
    n,p,d = image.shape
    
    r_lesion_values = image[:,:,0][image[:,:,0]*mask !=0]/255
    g_lesion_values = image[:,:,1][image[:,:,1]*mask !=0]/255
    b_lesion_values = image[:,:,2][image[:,:,2]*mask !=0]/255
    
    feat_df.loc[feat_df['ImageId'] == x, 'f18'] = np.percentile(r_lesion_values, 5, interpolation = 'lower')
    feat_df.loc[feat_df['ImageId'] == x, 'f19'] = np.percentile(r_lesion_values, 95, interpolation = 'lower')
    feat_df.loc[feat_df['ImageId'] == x, 'f20'] = np.mean(r_lesion_values)
    feat_df.loc[feat_df['ImageId'] == x, 'f21'] = np.var(r_lesion_values)
    
    feat_df.loc[feat_df['ImageId'] == x, 'f22'] = np.percentile(g_lesion_values, 5, interpolation = 'lower')
    feat_df.loc[feat_df['ImageId'] == x, 'f23'] = np.percentile(g_lesion_values, 95, interpolation = 'lower')
    feat_df.loc[feat_df['ImageId'] == x, 'f24'] = np.mean(g_lesion_values)
    feat_df.loc[feat_df['ImageId'] == x, 'f25'] = np.var(g_lesion_values)
    
    feat_df.loc[feat_df['ImageId'] == x, 'f26'] = np.percentile(b_lesion_values, 5, interpolation = 'lower')
    feat_df.loc[feat_df['ImageId'] == x, 'f27'] = np.percentile(b_lesion_values, 95, interpolation = 'lower')
    feat_df.loc[feat_df['ImageId'] == x, 'f28'] = np.mean(b_lesion_values)
    feat_df.loc[feat_df['ImageId'] == x, 'f29'] = np.var(b_lesion_values)
feat_df.to_csv('features.csv', index=None, sep=';', mode='w')
print("temps d'execution :", time()-t, "secondes")

temps d'execution : 58.56622505187988 secondes


## Classification
#### Importations

In [34]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import matthews_corrcoef 
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer

#### Formation de l'ensemble d'entrainement

In [38]:
# on importe les données
df = pd.read_csv('train.csv')
df_train_features = pd.merge(feat_df, df, on = 'ImageId')
#on enlève la dernière colonne car il s'agit de la classification et la première car il s'agit des Identifiants
X_train = df_train_features.iloc[:, 1:df_train_features.shape[1]-1]
Y_train = df_train_features.Malignant.values

#### On augmente les données

In [40]:
sm =SMOTE()
X_train, Y_train = sm.fit_sample(X_train, Y_train)

In [41]:
X_train.shape

(970, 29)

In [42]:
scale = StandardScaler()
X_train = scale.fit_transform(X_train)

On utilise GridSearchCV pour choisir les meilleurs paramètres possibles. On va répéter l'opération pour différents classifieur et regarder lequel donne le meilleur score.

In [87]:
grid_svc = [{'kernel': ['rbf'], 'gamma': [1e-1, 1e-2,1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

#on choisit le coefficient de corrélation de matthews en tant que score
estimator = GridSearchCV(SVC(), grid_svc, scoring = make_scorer(matthews_corrcoef))
estimator.fit(X_train, Y_train)
print(estimator.best_params_)
print("Score : ", estimator.best_score_)

{'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
Score :  0.821419398242


In [67]:
grid_RDF = {'max_depth': [10, 20, 25, 29], 'n_estimators': [20,30, 40, 50, 60, 70]}
            
#on choisit le coefficient de corrélation de matthews en tant que score
estimator = GridSearchCV(RandomForestClassifier(), grid_RDF, scoring = make_scorer(matthews_corrcoef))
estimator.fit(X_train, Y_train)
print(estimator.best_params_)
print("Score : ",estimator.best_score_)

{'max_depth': 29, 'n_estimators': 50}
Score :  0.765829196


In [68]:
grid_tree = {'max_depth': [10, 20, 25, 29]}

#on choisit le coefficient de corrélation de matthews en tant que score
estimator = GridSearchCV(DecisionTreeClassifier(), grid_tree, scoring = make_scorer(matthews_corrcoef))
estimator.fit(X_train, Y_train)
print(estimator.best_params_)
print("Score : ",estimator.best_score_)

{'max_depth': 10}
Score :  0.609455596214


In [69]:
grid_lda = {}

#on choisit le coefficient de corrélation de matthews en tant que score
estimator = GridSearchCV(LDA(), grid_lda, scoring = make_scorer(matthews_corrcoef))
estimator.fit(X_train, Y_train)
print("Score : ",estimator.best_score_)

Score :  0.463459616852


In [70]:
grid_qda = {}

#on choisit le coefficient de corrélation de matthews en tant que score
estimator = GridSearchCV(QDA(), grid_qda, scoring = make_scorer(matthews_corrcoef))
estimator.fit(X_train, Y_train)
print("Score : ",estimator.best_score_)

Score :  0.536026074001


C'est SVM avec un noyau gaussien, C = 100 et gamma = 0.1 qui semble être le meilleur classifieur car il obtient un score plus élevé que les autres.

In [99]:
clf = SVC(kernel='rbf', C= 100, gamma=0.1)
clf.fit(X_train, Y_train)
# on ouvre le fichier où les identifiants des photos de test sont stockées
df_submission = pd.read_csv('test.csv', sep=',')

for Id in df_submission['ImageId']:
    val = feat_df.loc[feat_df['ImageId'] == Id].values[0, 1:]
    # il faut centrer et réduire les données de test aussi
    df_submission.loc[df_submission['ImageId'] == Id, 'Malignant'] = clf.predict(scale.transform(np.array([val])))
    
df_submission['Malignant'] = df_submission['Malignant'].astype(int)
# on enregistre la prédiction
df_submission.to_csv('test.csv', index=None, sep=',', mode='w') 

/Users/matthis/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


On obtient un score de 0.23431

In [96]:
pipe = Pipeline(steps=[('pca', PCA()), ('svc', SVC(kernel ='rbf'))])
estimator = GridSearchCV(pipe, dict(pca__n_components = [21,22,23,24,25,26,27,28, 29], svc__C = [100], svc__gamma=[0.1]), scoring = make_scorer(matthews_corrcoef))
estimator.fit(X_train, Y_train)

print(estimator.best_params_)
print(estimator.best_score_)

{'pca__n_components': 23, 'svc__C': 100, 'svc__gamma': 0.1}
0.821515933873


L'erreur est quasiment la même qu'avec 29 features. De plus, lorsque l'on fait la prediction avec l'estimateur calculé, on obtient exactement le même fichier que si l'on avait pas fait la PCA. On ne va donc pas appliquer la PCA.